In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import os
import json
import datetime


## Functions for scrapping the website

In [79]:
def get_subcategories(link):
    subPage = requests.get(link)
    subSoup = BeautifulSoup(subPage.content, 'html.parser')
    filterSection = subSoup.find('div', id='search_filters').find('div', class_='tvserach-filter-wrapper').find('div', class_='tvsearch-filter-content-wrapper').find_all('section')
    subCateg = {}
    subCategNoLinks = {}
    for section in filterSection:
        name = section.find('p').text
        subCateg[name] = []
        subCategNoLinks[name] = []
        subcategories = section.find('ul', class_='tvfilter-search-types-dropdown').find_all('li')
        if  (section.find('ul', class_='tvfilter-search-types-dropdown').has_attr('data-slider-min')) == False:
            for subcategory in subcategories:
                subCateg[name].append([subcategory.find('a').text.split('\n')[1].strip(), subcategory.find('a')['href']])
                subCategNoLinks[name].append(subcategory.find('a').text.split('\n')[1].strip())
        else:
            subPriceMax = section.find('ul', class_='tvfilter-search-types-dropdown')['data-slider-max']
            subPriceMin = section.find('ul', class_='tvfilter-search-types-dropdown')['data-slider-min']
            subCateg[name].append(subPriceMin)
            subCateg[name].append(subPriceMax)
            subCategNoLinks[name].append(subPriceMin)
            subCategNoLinks[name].append(subPriceMax)
    return subCateg, subCategNoLinks


In [80]:
website = 'https://www.lamode.tn/'
#create a dictionary to store the data
data = {}
data['categories'] = []
dataNoLink = {}
dataNoLink['categories'] = []
#extract categories and their links using beautiful soup
def get_categories():
    #get the page
    page = requests.get(website)
    #parse it
    soup = BeautifulSoup(page.content, 'html.parser')
    categories = soup.find('div',class_='container-fluid main-categories-home').find('div',class_='row').find_all('div',class_='row')
    elementsLis = []
    for a in categories:
        #get only the direct children of a
        elems = a.findChildren(recursive=False)
        for elem in elems:
            elementsLis.append(elem)
    #loop through the categories
    for category in elementsLis:
        #get the link and the name of the category
        link = category.find('a')['href']
        name = category.find('div', class_="tvcategory-slider-info-box").text
        #print(name)
        #get the subcategories
        subcategories, subcategoriesNoLink = get_subcategories(link)
        #add the category to the dictionary
        data['categories'].append({
            'name': name,
            'link': link,
            'subcategories': subcategories
        })
        dataNoLink['categories'].append({
            'name': name,
            'subcategories': subcategoriesNoLink
        })
    return data, dataNoLink
    
dict, NoLink = get_categories()



In [115]:
NoLink

{'categories': [{'name': 'Montres',
   'subcategories': {'Catégories': ['Maintenance',
     'Montres pour Enfants',
     'Montres pour Femmes',
     'Montres pour Hommes',
     'Smartwatches'],
    'Marque': ['ALBERTO RICCI',
     'ARMANI EXCHANGE',
     'Benyar',
     'CALVIN KLEIN',
     'Casio',
     'Curren',
     'Daniel Wellington',
     'Diesel',
     'Dream',
     'Emporio Armani',
     'ESPRIT',
     'Festina',
     'Fossil',
     'Guess',
     'Hugo boss',
     'MASERATI',
     'MEGIR',
     'Michael kors',
     'MINI FOCUS',
     'PASNEW',
     'SANDA',
     'SKMEI',
     'sk shengke',
     'Tissot'],
    'Prix': ['9', '1980']}},
  {'name': 'Lunettes',
   'subcategories': {'Catégories': ['Cadres Optiques',
     'Lentilles de contact',
     'Lunettes de Soleil'],
    'Marque': ['9.81EYEWEAR',
     'ADIDAS',
     'ALBERTO RICCI',
     'Ana Hickmanni',
     'Burberry',
     'BVLGARI',
     'CALVIN KLEIN',
     'Carrera',
     'CELINE',
     'Chloé',
     'Diesel',
     'Dior',


## saving the data in a json file

In [110]:
#save the data in a json file
with open('WebsiteCategoriesData.json', 'w') as outfile:
    json.dump(dict, outfile)
with open('WebsiteCategoriesDataNoLink.json', 'w') as outfile:
    json.dump(NoLink, outfile)

## Test Data (manual)

In [112]:
TestData = """Query: Je recherche une montre pour hommes de marque Tissot avec un prix inférieur à 1000DT.
Category: montre
Subcategory: montre pour hommes
Marque: Tissot
Prix: Inférieur à 1000DT

Query: Je veux acheter des lunettes de soleil de marque Ray-Ban avec un prix entre 9 et 1420DT.
Category: lunettes
Subcategory: Lunettes de Soleil
Marque: Ray-Ban
Prix: Entre 9 et 1420DT

Query: Je cherche un parfum pour femmes de la marque Chanel avec un volume de 50ml.
Category: parfum
Subcategory: parfum pour Femmes
Marque: Chanel
Volume: 50ml

Query: Je souhaite trouver un sac de la marque JEEP avec un prix inférieur à 199DT.
Category: sac
Marque: JEEP
Prix: Inférieur à 199DT

Query: Je veux acheter des chaussures pour hommes de la marque Stephan avec une pointure de 43.
Category: Chaussures
Subcategory: Chaussures Hommes
Marque: Stephan
Pointure: 43

Query: Je recherche des bijoux de la marque Yves Saint Laurent avec un prix entre 15 et 1588DT.
Category: Accessoires
Subcategory: Bijoux
Marque: Yves Saint Laurent
Prix: Entre 15 et 1588DT

Query: Je veux acheter un soin anti-âge de la marque Clarins avec un volume de 50ml.
Category: Soins
Subcategory: Anti Âge
Marque: Clarins
Volume: 50ml

Query: Je souhaite trouver un rouge à lèvres de la marque MAC avec un prix inférieur à 319DT.
Category: Maquillage
Subcategory: Lèvres
Marque: MAC
Prix: Inférieur à 319DT

Query: Je recherche une montre pour hommes de marque Tissot avec un prix inférieur à 1000DT.
Category: Montres
Subcategory: Montres pour Hommes
Marque: Tissot
Prix: Inférieur à 1000DT

Query: Je veux acheter des lunettes de soleil de marque Ray-Ban avec un prix entre 9 et 1420DT.
Category: Lunettes
Subcategory: Lunettes de Soleil
Marque: Ray-Ban
Prix: Entre 9 et 1420DT

Query: Je cherche un parfum pour femmes de la marque Chanel avec un volume de 50ml.
Category: Parfum
Subcategory: Parfum pour Femmes
Marque: Chanel
Volume: 50ml

Query: Je souhaite trouver un sac de la marque JEEP avec un prix inférieur à 199DT.
Category: Sacs
Marque: JEEP
Prix: Inférieur à 199DT

Query: Je veux acheter des chaussures pour hommes de la marque Stephan avec une pointure de 43.
Category: Chaussures
Subcategory: Chaussures Hommes
Marque: Stephan
Pointure: 43

Query: Je recherche des bijoux de la marque Yves Saint Laurent avec un prix entre 15 et 1588DT.
Category: Accessoires
Subcategory: Bijoux
Marque : Yves Saint Laurent
Prix : Entre 15 et 1588DT

Query: Je veux acheter un soin anti-âge de la marque Clarins avec un volume de 50ml.
Category: Soins
Subcategory: Anti Âge
Marque: Clarins
Volume: 50ml

Query: Je souhaite trouver un rouge à lèvres de la marque MAC avec un prix inférieur à 319DT.
Category: Maquillage
Subcategory: Lèvres
Marque: MAC
Prix: Inférieur à 319DT"""

## transforming the data into a savable format (json) and saving it in a file

In [113]:
def transform_to_ner_dataset(query):
    ner_dataset = []
    
    parts = query.split("\n\n")
    
    for part in parts:
        lines = part.split("\n")
        entity_dict = {}
        
        for line in lines:
            if line.startswith("Query:"):
                text = line.split("Query:")[1].strip()
            else:
                key, value = line.split(":")
                entity_type = key.strip()
                entity_value = value.strip()
                
                entity_dict[entity_type] = {
                    "entity": entity_type,
                    "value": entity_value,
                }
        entities = list(entity_dict.values())
        ner_data = {
            "text": text,
            "entities": entities
        }
        ner_dataset.append(ner_data)
    return ner_dataset


In [114]:
dataset= transform_to_ner_dataset(TestData)
with open('TestData.json', 'w') as outfile:
    json.dump(dataset, outfile)

In [77]:
print(dict)

{'categories': [{'name': 'Montres', 'link': 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie', 'subcategories': {'Catégories': [['Maintenance', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Cat%C3%A9gories-Maintenance'], ['Montres pour Enfants', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Cat%C3%A9gories-Montres+pour+Enfants'], ['Montres pour Femmes', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Cat%C3%A9gories-Montres+pour+Femmes'], ['Montres pour Hommes', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Cat%C3%A9gories-Montres+pour+Hommes'], ['Smartwatches', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Cat%C3%A9gories-Smartwatches']], 'Marque': [['ALBERTO RICCI', 'https://www.lamode.tn/13-montre-pour-homme-et-femme-aux-meilleurs-prix-en-tunisie?q=Marque-ALBE